In [16]:
# Dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
import os
import glob
from IPython.core.display import HTML
import reverse_geocoder as rg


#Oklahoma Latitude and Longitude Limits
minlatitude = 33.863,
maxlatitude = 37.004,
minlongitude = -99.976,
maxlongitude = -94.559

url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
payload = {
    "format" : "geojson",
    "minlatitude" : minlatitude,
    "maxlatitude" : maxlatitude,
    "minlongitude" : minlongitude,
    "maxlongitude" : maxlongitude,
    "starttime" : "2013-01-01",
    "endtime" : "2018-12-31",
    "minmagnitude" : 2
    
}

In [49]:
# List out the files needed using the glob module
excel_data = glob.glob("Injection_Volumes_Data/*.xlsx")
excel_data

['Injection_Volumes_Data/2015UICvolumes.xlsx',
 'Injection_Volumes_Data/2013UICvolumes.xlsx',
 'Injection_Volumes_Data/2014UICvolumes.xlsx',
 'Injection_Volumes_Data/2016UICvolumes.xlsx',
 'Injection_Volumes_Data/2018UICvolumes.xlsx',
 'Injection_Volumes_Data/2017UICvolumes.xlsx']

In [50]:
# Import all the files and combine them into one dataframe
all_data_df = pd.DataFrame()
for f in excel_data:
    df = pd.read_excel(f)
    all_data_df = all_data_df.append(df,ignore_index=True, sort=False)

In [53]:
all_data_df

,API,OperatorName,WellName,WellNumber,Lat_Y,Long_X,CountyName,ReportYear,TotalDepth,Jan Vol,...,Mar Vol,Apr Vol,May Vol,Jun Vol,Jul Vol,Aug Vol,Sep Vol,Oct Vol,Nov Vol,Dec Vol
0,3500300026,PHOENIX PETROCORP INC,SE EUREKA UNIT-TUCKER #1,21,36.900324,-98.21826,ALFALFA,2015,5012.0,465.0,...,465.0,450.0,465.0,450.0,465.0,465.0,450.0,465.0,411.0,404.0
1,3500300163,CHAMPLIN EXPLORATION INC,CHRISTENSEN,1,36.896636,-98.17772,ALFALFA,2015,5060.0,1063.0,...,1063.0,1063.0,1063.0,1063.0,1063.0,1063.0,1063.0,1063.0,1063.0,992.0
2,3500320145,CONTINENTAL RESOURCES INC,SINGREE,1,36.504849,-98.43324,ALFALFA,2015,6700.0,4804.0,...,3789.0,3444.0,360.0,1460.0,1342.0,1586.0,1546.0,300.0,0.0,0.0
3,3500320145,CONTINENTAL RESOURCES INC,SINGREE,1,36.504849,-98.43324,ALFALFA,2015,6700.0,4804.0,...,3789.0,3444.0,360.0,1460.0,1342.0,1586.0,1546.0,300.0,0.0,0.0
4,3500320786,LINN OPERATING LLC,NE CHEROKEE UNIT,85,36.806113,-98.32584,ALFALFA,2015,5247.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70552,35153235940000,SANDRIDGE EXPLORATION & PRODUCTION LLC,GARVIE SWD,1,0.000000,0.00000,WOODWARD,2017,5300.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70553,35153235940000,SANDRIDGE EXPLORATION & PRODUCTION LLC,GARVIE SWD,1,0.000000,0.00000,WOODWARD,2017,5300.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70554,35153235940000,SANDRIDGE EXPLORATION & PRODUCTION LLC,GARVIE SWD,1,0.000000,0.00000,WOODWARD,2017,5300.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70555,35153235940000,SANDRIDGE EXPLORATION & PRODUCTION LLC,GARVIE SWD,1,0.000000,0.00000,WOODWARD,2017,5300.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
#Get total volumes per month per WellName
total_volume_per_month = all_data_df.groupby(["WellName", "Lat_Y", "Long_X", "CountyName", "TotalDepth", "ReportYear"])["Jan Vol", "Feb Vol", "Mar Vol", 
                                                           "Apr Vol", "May Vol", "Jun Vol", 
                                                           "Jul Vol", "Aug Vol", "Sep Vol", 
                                                           "Oct Vol", "Nov Vol", "Dec Vol"].sum()

In [56]:
total_volume_per_month

Jan Vol  \
WellName                    Lat_Y     Long_X     CountyName TotalDepth ReportYear            
 CALDWELL-LIONEL HARRIS     36.074014 -96.766570 PAYNE      3920.0     2013         1500.0   
                                                                       2014         1500.0   
                                                                       2015         1500.0   
                                                                       2016          960.0   
 MARTHA HOLATHA  AKA WACOHO 35.990140 -96.024770 TULSA      0.0        2013         1349.0   
...                                                                                    ...   
ZOMA TANNER                 36.771194 -95.552583 NOWATA     696.0      2014         1581.0   
                                                                       2015         1760.0   
                                                                       2016         1610.0   
                                                                       2017         1593.0   
                                                                       2018         1512.0   

                                                                                   Feb Vol  \
WellName                    Lat_Y     Long_X     CountyName TotalDepth ReportYear            
 CALDWELL-LIONEL HARRIS     36.074014 -96.766570 PAYNE      3920.0     2013         1500.0   
                                                                       2014         1500.0   
                                                                       2015         1500.0   
                                                                       2016          960.0   
 MARTHA HOLATHA  AKA WACOHO 35.990140 -96.024770 TULSA      0.0        2013         1216.0   
...                                                                                    ...   
ZOMA TANNER                 36.771194 -95.552583 NOWATA     696.0      2014         1764.0   
                                                                       2015         1382.0   
                                                                       2016         1479.0   
                                                                       2017         1410.0   
                                                                       2018            0.0   

                                                                                   Mar Vol  \
WellName                    Lat_Y     Long_X     CountyName TotalDepth ReportYear            
 CALDWELL-LIONEL HARRIS     36.074014 -96.766570 PAYNE      3920.0     2013         1500.0   
                                                                       2014         1500.0   
                                                                       2015         1500.0   
                                                                       2016          960.0   
 MARTHA HOLATHA  AKA WACOHO 35.990140 -96.024770 TULSA      0.0        2013         1329.0   
...                                                                                    ...   
ZOMA TANNER                 36.771194 -95.552583 NOWATA     696.0      2014         1489.0   
                                                                       2015         1987.0   
                                                                       2016         1632.0   
                                                                       2017         1431.0   
                                                                       2018         1643.0   

                                                                                   Apr Vol  \
WellName                    Lat_Y     Long_X     CountyName TotalDepth ReportYear            
 CALDWELL-LIONEL HARRIS     36.074014 -96.766570 PAYNE      3920.0     2013         1500.0   
                                                                       2014         1500.0   
                                      

In [57]:
#Convert to dataframe 
total_volume_per_month_df = pd.DataFrame(total_volume_per_month).reset_index()
total_volume_per_month_df

,WellName,Lat_Y,Long_X,CountyName,TotalDepth,ReportYear,Jan Vol,Feb Vol,Mar Vol,Apr Vol,May Vol,Jun Vol,Jul Vol,Aug Vol,Sep Vol,Oct Vol,Nov Vol,Dec Vol
0,CALDWELL-LIONEL HARRIS,36.074014,-96.766570,PAYNE,3920.0,2013,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
1,CALDWELL-LIONEL HARRIS,36.074014,-96.766570,PAYNE,3920.0,2014,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
2,CALDWELL-LIONEL HARRIS,36.074014,-96.766570,PAYNE,3920.0,2015,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
3,CALDWELL-LIONEL HARRIS,36.074014,-96.766570,PAYNE,3920.0,2016,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0
4,MARTHA HOLATHA AKA WACOHO,35.990140,-96.024770,TULSA,0.0,2013,1349.0,1216.0,1329.0,1192.0,1299.0,1365.0,1389.0,1341.0,1250.0,1577.0,1303.0,1284.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58891,ZOMA TANNER,36.771194,-95.552583,NOWATA,696.0,2014,1581.0,1764.0,1489.0,1701.0,1981.0,1631.0,1499.0,1893.0,1510.0,1601.0,1415.0,1839.0
58892,ZOMA TANNER,36.771194,-95.552583,NOWATA,696.0,2015,1760.0,1382.0,1987.0,1979.0,1602.0,1897.0,1767.0,1510.0,1493.0,1601.0,1431.0,1397.0
58893,ZOMA TANNER,36.771194,-95.552583,NOWATA,696.0,2016,1610.0,1479.0,1632.0,1735.0,1469.0,1421.0,1492.0,1374.0,0.0,1401.0,1473.0,1632.0
58894,ZOMA TANNER,36.771194,-95.552583,NOWATA,696.0,2017,1593.0,1410.0,1431.0,1410.0,1518.0,1431.0,1493.0,1431.0,1620.0,1823.0,923.0,1632.0


In [65]:
#Delete data with zero volume (pick non-zero values which turns all zero values into nan and remove nan-values)
clean_total_volume_per_month_df = total_volume_per_month_df[total_volume_per_month_df!=0].dropna()

In [66]:
clean_total_volume_per_month_df

,WellName,Lat_Y,Long_X,CountyName,TotalDepth,ReportYear,Jan Vol,Feb Vol,Mar Vol,Apr Vol,May Vol,Jun Vol,Jul Vol,Aug Vol,Sep Vol,Oct Vol,Nov Vol,Dec Vol
0,CALDWELL-LIONEL HARRIS,36.074014,-96.766570,PAYNE,3920.0,2013,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
1,CALDWELL-LIONEL HARRIS,36.074014,-96.766570,PAYNE,3920.0,2014,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
2,CALDWELL-LIONEL HARRIS,36.074014,-96.766570,PAYNE,3920.0,2015,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
3,CALDWELL-LIONEL HARRIS,36.074014,-96.766570,PAYNE,3920.0,2016,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0,960.0
29,(TDU) CD,34.509226,-97.533300,GARVIN,2866.0,2013,23357.0,27410.0,31970.0,30890.0,30271.0,29169.0,31720.0,28963.0,22239.0,24269.0,24379.0,23565.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58885,ZOMA TANNER,36.769111,-95.551333,NOWATA,697.0,2014,9989.0,8631.0,8241.0,10762.0,10404.0,13732.0,12640.0,12637.0,9927.0,11869.0,3665.0,12365.0
58890,ZOMA TANNER,36.771194,-95.552583,NOWATA,696.0,2013,1680.0,1710.0,2101.0,2091.0,2231.0,1503.0,1581.0,1981.0,2131.0,1510.0,1733.0,1831.0
58891,ZOMA TANNER,36.771194,-95.552583,NOWATA,696.0,2014,1581.0,1764.0,1489.0,1701.0,1981.0,1631.0,1499.0,1893.0,1510.0,1601.0,1415.0,1839.0
58892,ZOMA TANNER,36.771194,-95.552583,NOWATA,696.0,2015,1760.0,1382.0,1987.0,1979.0,1602.0,1897.0,1767.0,1510.0,1493.0,1601.0,1431.0,1397.0


In [68]:
#Get location of injection wells
wells_location = clean_total_volume_per_month_df.pivot_table(values=["Lat_Y", "Long_X"], index=("WellName"))
wells_location

,Lat_Y,Long_X
WellName,,
CALDWELL-LIONEL HARRIS,36.074014,-96.766570
(TDU) CD,34.509226,-97.533300
(TDU) RI,34.508796,-97.527780
(TDU) SH,34.512399,-97.543700
/SUE RICHARDSON/EVELYN C,34.522590,-99.234210
...,...,...
ZIMMERMAN SWD (1A),35.904161,-98.434260
ZIPSIE,34.435691,-98.268090
ZOLDOSKE,36.421100,-97.364211


In [135]:
#Get number of wells in each county per WellName
county_wells_count = clean_total_volume_per_month_df.groupby(["WellName","CountyName"])["CountyName"].count()
county_wells_df = pd.DataFrame(county_wells_count)
county_wells_df.columns = ["NumberofWells"]
county_wells_df.reset_index()

,WellName,CountyName,NumberofWells
0,CALDWELL-LIONEL HARRIS,PAYNE,4
1,(TDU) CD,GARVIN,6
2,(TDU) RI,GARVIN,6
3,(TDU) SH,GARVIN,6
4,/SUE RICHARDSON/EVELYN C,JACKSON,3
...,...,...,...
6746,ZIMMERMAN SWD (1A),BLAINE,1
6747,ZIPSIE,COTTON,6
6748,ZOLDOSKE,NOBLE,2
6749,ZOLLAR,LINCOLN,5


In [137]:
#Get total wells per county
total_wells_per_county = county_wells_df.groupby(["CountyName"])
total_wells_per_county.size()

CountyName
ALFALFA        95
BEAVER         84
BECKHAM        19
BLAINE         42
BRYAN           5
             ... 
WAGONER        19
WASHINGTON    162
WASHITA         6
WOODS          81
WOODWARD       16
Length: 66, dtype: int64